In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.0'

# Classifying traffic conditions or hazards: a multi-class classification example


In [5]:
import os, shutil, random

In [6]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = r'./trafficnet_dataset_v1'


train_dir = os.path.join(original_dataset_dir, 'train')
validation_dir = os.path.join(original_dataset_dir, 'validation')
test_dir = os.path.join(original_dataset_dir, 'test')


accident_train_dir = os.path.join(train_dir, 'accident')
dense_traffic_train_dir = os.path.join(train_dir, 'dense_traffic')
fire_train_dir = os.path.join(train_dir, 'fire')
sparse_traffic_train_dir = os.path.join(train_dir, 'sparse_traffic')

accident_test_dir = os.path.join(test_dir, 'accident')
dense_traffic_test_dir = os.path.join(test_dir, 'dense_traffic')
fire_test_dir = os.path.join(test_dir, 'fire')
sparse_traffic_test_dir = os.path.join(test_dir, 'sparse_traffic')

In [5]:

os.mkdir(validation_dir)
# Directory with our validation accident pictures
validation_accident_dir = os.path.join(validation_dir, 'accident')
os.mkdir(validation_accident_dir)
validation_dense_traffic_dir = os.path.join(validation_dir, 'dense_traffic')
os.mkdir(validation_dense_traffic_dir)
validation_fire_dir = os.path.join(validation_dir, 'fire')
os.mkdir(validation_fire_dir)
validation_sparse_traffic_dir = os.path.join(validation_dir, 'sparse_traffic')
os.mkdir(validation_sparse_traffic_dir)



In [7]:
print('total training accident images:', len(os.listdir(accident_train_dir)))
print('total training dense traffic images:', len(os.listdir(dense_traffic_train_dir)))
print('total training fire images:', len(os.listdir(fire_train_dir)))
print('total training sparse traffic images:', len(os.listdir(sparse_traffic_train_dir)))
print("\n")
print('total test accident images:', len(os.listdir(accident_test_dir)))
print('total test dense traffic images:', len(os.listdir(dense_traffic_test_dir)))
print('total test fire images:', len(os.listdir(fire_test_dir)))
print('total test sparse traffic images:', len(os.listdir(sparse_traffic_test_dir)))

FileNotFoundError: [WinError 3] The system cannot find the path specified: './trafficnet_dataset_v1\\train\\accident'

In [7]:
#Taking a 80:20 split of training and validation data

validation_accident_len = int(len(os.listdir(accident_train_dir)) * 0.2)
print('validation_accident_len', validation_accident_len)

validation_dense_traffic_len = int(len(os.listdir(dense_traffic_train_dir)) * 0.2)
print('validation_dense_traffic_len', validation_dense_traffic_len)
validation_fire_len = int(len(os.listdir(fire_train_dir)) * 0.2)
print('validation_fire_len', validation_fire_len)
validation_sparse_traffic_len = int(len(os.listdir(sparse_traffic_train_dir)) * 0.2)
print('validation_sparse_traffic_len', validation_sparse_traffic_len)




validation_accident_len 180
validation_dense_traffic_len 180
validation_fire_len 180
validation_sparse_traffic_len 180


In [8]:
# Adding images to validation directory
training_accident_files = os.listdir(accident_train_dir)
training_dense_traffic_files = os.listdir(dense_traffic_train_dir)
training_fire_files = os.listdir(fire_train_dir)
training_sparse_traffic_files = os.listdir(sparse_traffic_train_dir)
#print( random.sample(training_accident_files,validation_accident_len ))


for accident_img_file in random.sample(training_accident_files,validation_accident_len ):
    srcFile = os.path.join(accident_train_dir,  accident_img_file)
    shutil.move(srcFile,validation_accident_dir )
    
for dense_traffic_img_file in random.sample(training_dense_traffic_files,validation_dense_traffic_len ):
    srcFile = os.path.join(dense_traffic_train_dir,  dense_traffic_img_file)
    shutil.move(srcFile,validation_dense_traffic_dir )
    
for fire_img_file in random.sample(training_fire_files,validation_fire_len ):
    srcFile = os.path.join(fire_train_dir ,  fire_img_file)
    shutil.move(srcFile,validation_fire_dir )
    
for sparse_traffic_img_file in random.sample(training_sparse_traffic_files,validation_sparse_traffic_len ):
    srcFile = os.path.join(sparse_traffic_train_dir,  sparse_traffic_img_file)
    shutil.move(srcFile,validation_sparse_traffic_dir )  


In [8]:
print('total validation accident images:', len(os.listdir(validation_accident_dir)))
print('total validation dense traffic images:', len(os.listdir(validation_dense_traffic_dir)))
print('total validation fire images:', len(os.listdir(validation_fire_dir)))
print('total validation sparse traffic images:', len(os.listdir(validation_sparse_traffic_dir)))

NameError: name 'validation_accident_dir' is not defined

## Building our network


In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(150, 150, 3)))
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(4, activation='softmax'))

Using TensorFlow backend.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4320064   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
Total params: 4,396,484
Trainable params: 4,396,484
Non-trainable params: 0
_________________________________________________________________


Compilation, loss function and optimiser selection

In [3]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Data preprocessing


In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)
validate_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=30,
        class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=30,
        class_mode='categorical')

NameError: name 'train_dir' is not defined

In [78]:

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (30, 150, 150, 3)
labels batch shape: (30, 4)


## Model Fitting

In [79]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=96,
      epochs=24,
      validation_data=validation_generator,
      validation_steps=24)

Epoch 1/30
96/96 [==============================] - 17s 178ms/step - loss: 1.3567 - accuracy: 0.3139 - val_loss: 1.4226 - val_accuracy: 0.2944
Epoch 2/30
96/96 [==============================] - 17s 172ms/step - loss: 1.3039 - accuracy: 0.3521 - val_loss: 1.4534 - val_accuracy: 0.3486
Epoch 3/30
96/96 [==============================] - 16s 167ms/step - loss: 1.2574 - accuracy: 0.4122 - val_loss: 1.2459 - val_accuracy: 0.3722
Epoch 4/30
96/96 [==============================] - 19s 199ms/step - loss: 1.2352 - accuracy: 0.4153 - val_loss: 1.2200 - val_accuracy: 0.3903
Epoch 5/30
96/96 [==============================] - 15s 155ms/step - loss: 1.2197 - accuracy: 0.4274 - val_loss: 1.2329 - val_accuracy: 0.4319
Epoch 6/30
96/96 [==============================] - 16s 164ms/step - loss: 1.2123 - accuracy: 0.4389 - val_loss: 1.1262 - val_accuracy: 0.4417
Epoch 7/30
96/96 [==============================] - 16s 167ms/step - loss: 1.2070 - accuracy: 0.4368 - val_loss: 1.3100 - val_accuracy: 0.3944

In [80]:
model.save('PredictingTrafficConditionsHazards_convLayerIncrease.h5')

## Plotting Training and validation Accuracy and loss

In [81]:
tester_datagen = ImageDataGenerator(rescale=1./255)

test_generator = tester_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=1,
        class_mode='categorical',
        shuffle = 'False')

results = model.evaluate_generator(test_generator,workers=6,steps=800 )

Found 800 images belonging to 4 classes.


In [82]:
print(results)

[1.2533209323883057, 0.5024999976158142]


In [83]:
model.save('PredictingTrafficConditionsHazards_2.h5')